In [1]:
from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.aa import AA_TEST
from hypex.reporters import AADictReporter

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.data.dtypes

user_id            int32
signup_month       int32
treat              int32
pre_spends       float64
post_spends      float64
age              float64
gender          category
industry        category
dtype: object

In [4]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column_name] = self.data[column_name].astype(type_name)
C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column_name] = self.data[column_name].astype(type_name)
C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:189: SettingWithCopyWarning: 
A value is trying

In [5]:
result.analysis_tables

{'GroupSizes┴┴':                      B
 control size    5000.0
 control size %    50.0
 test size       5000.0
 test size %       50.0,
 'GroupDifference┴┴':                                    B
 post_spends control mean  452.292822
 post_spends difference     -0.256533
 post_spends difference %   -0.056718
 post_spends test mean     452.036289,
 'TTest┴┴':   group  statistic   p-value   pass
 0     B   0.325501  0.744809  False,
 'KSTest┴┴':   group  statistic   p-value   pass
 0     B      0.026  0.068089  False,
 'OneAAStatAnalyzer┴┴':    TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
 0       0.744809         0.0        0.068089          0.0         0.293663}

In [6]:
AADictReporter().report(result)

{'random_state': None,
 'B post_spends control mean': 452.2928222222222,
 'B post_spends difference': -0.256533333333266,
 'B post_spends difference %': -0.056718417964907175,
 'B post_spends test mean': 452.0362888888889,
 'B control size': 5000.0,
 'B control size %': 50.0,
 'B test size': 5000.0,
 'B test size %': 50.0,
 'TTest p-value': 0.7448091152919443,
 'TTest pass': 0.0,
 'KSTest p-value': 0.0680893184951445,
 'KSTest pass': 0.0,
 'mean test score': 0.29366258409407775}